<h1 align="Center">Retrieving PI Point Data</h1>
In this notebook, we will explore the various ways to retrieve PI Point data using python and the AF SDK. 

<hr>
We will start off by loading in the AF SDK. Check out the previous notebook "1. Starting with AF SDK and Python" if you dont understand what this code does!

In [3]:
# pythonnet is imported as "clr"
# this package allows you to import dotNet DLL's and run them within your python environment
import clr
import os
import sys

# Get the installation directory from the environment variable or fall back to the Windows
# default installation path
PIAF_SDK = os.getenv('PIHOME', 'C:\\Program Files\\PIPC')
PIAF_SDK += '\\AF\\PublicAssemblies\\4.0\\'

# check if the AF SDK default directories exist
if not os.path.isdir(PIAF_SDK):
    raise ImportError('PIAF SDK not found in {}, check installation'.format(PIAF_SDK))
    
# append them to the PATH environment variable
sys.path.append(PIAF_SDK)

# add the AF SDK DLL 
clr.AddReference('OSIsoft.AFSDK')

# import the AF SDK
from OSIsoft import AF

print(AF)

<module 'OSIsoft.AF'>


In [4]:
# get first item in GetPIServers object
piserver = AF.PI.PIServers.GetPIServers()[0]

# search for pi point on the pi data archive
pipoint = AF.PI.PIPoint.FindPIPoint(piserver, "sinusoid")

print("Found {} on data archive {}".format(pipoint.Name, piserver.Name))

Found SINUSOID on data archive DSF-PIDA1


<h3>PI Point Attributes</h3>
Let examine some properties of the PIPoint object.

In [ ]:
# print immutable properties
print(pipoint.ID)
print(pipoint.PointClass)
print(pipoint.PointType)
print(pipoint.Server)
print(pipoint.Step)

You will notice there are many "properties" missing *(e.g. PointSource, InsturmentTag, Location1, Location2, ...)*. The PIPoint object actually does not consider those properties, but instead, <em><b>attributes</b></em>. The PIPoint Attributes list can be found using the <b>FindAttributeNames</b> method.

In [ ]:
pipoint.FindAttributeNames("*")

<hr>
<h3>Retrieving PIPoint Values</h3>

In [38]:
interpolated_value = pipoint.InterpolatedValue('*-1h')

TypeError: No method matches given arguments for InterpolatedValue

Hmmm that didn't seem to work... This sh\*t is broken! Let's take a breath and read the error message.
<blockquote><font color="maroon"><b>TypeError</b></font>: No method matches given arguments for InterpolatedValue</blockquote>
The TypeError specifies that we have an incorrect object Type somewhere.. and the error message tells us exactly where. The <a href="https://techsupport.osisoft.com/Documentation/PI-AF-SDK/html/M_OSIsoft_AF_PI_PIPoint_InterpolatedValue.htm">InterpolatedValue</a> method does not have a parameter set that matches a String object as input.
<img src="./img/interpolated_value.png" />
<hr>
Looking at the documentation, it specifies the input object must be an AFTime object. Even more, it provides the namespace of the object <em>AF.Time.AFTime</em>. Let's try casting our time string into an AFTime object.

<hr>
<h3>PIPoint.InterpolatedValue Method</a>

In [39]:
interpolated_value = pipoint.InterpolatedValue(AF.Time.AFTime('*-1h'))
print(interpolated_value)

78.97469


Success! Let's take a look at the returned <em>interpolated_value</em> variable.

In [8]:
type(interpolated_value)

OSIsoft.AF.Asset.AFValue

Interestingly, it is not a float or string data type, but actually an AFValue object!
<hr>
Let's investigate what type of properties this object has within python. As always, a list of all the properties can be found in the AF SDK documentation for <a href="https://techsupport.osisoft.com/Documentation/PI-AF-SDK/Html/T_OSIsoft_AF_Asset_AFValue.htm">AFValue</a>.

In [17]:
print(interpolated_value.Timestamp)
print(interpolated_value.Attribute)
print(interpolated_value.IsGood)
print(interpolated_value.UOM)
print(interpolated_value.Value)
print(interpolated_value.ValueType)

5/5/2020 3:54:51 PM
None
True
None
92.63043
System.Single


<hr>
<h3>PIPoint.RecordedValue Method</h3>
Let's try a different method to retrieve PI data, the <a href="https://techsupport.osisoft.com/Documentation/PI-AF-SDK/html/M_OSIsoft_AF_PI_PIPoint_RecordedValue.htm">RecordedValue</a> method. This method will return *only* recorded values.

In [7]:
recorded_value = pipoint.RecordedValue(AF.Time.AFTime('*-1h'))
print("Timestamp: {} | Value: {}".format(recorded_value.Timestamp, recorded_value.Value))

Timestamp: 5/5/2020 3:55:13 PM | Value: 92.5471


Notice that both of the above methods return only a *single* value. What if we want to return many values of a PI Point?
To do this, we will use the InterpolatedValues, RecordedValues, and PlotValues methods.
<hr>
<h3>PIPoint.InterpolatedValues Method</h3>

In [ ]:
interpolated_values = pipoint.InterpolatedValues(
    AF.Time.AFTimeRange('*-1h','*'), AF.Time.AFTimeSpan(0, 0, 0, 0.0, 5.0), '', True)
print([val.Value for val in interpolated_values])

<hr>
<h3>PIPoint.RecordedValues Method</h3>
The RecordedValues method requires five parameters, described below in C# code. We must instantiate objects of each type to execute the method.
<blockquote><font color="blue">public</font> AFValues RecordedValues(<br>
    &emsp;&emsp;AFTimeRange <em>timeRange</em>,<br>
	&emsp;&emsp;AFBoundaryType <em>boundaryType</em>,<br>
	&emsp;&emsp;string <em>filterExpression</em>,<br>
	&emsp;&emsp;bool <em>includeFilteredValues</em>,<br>
	&emsp;&emsp;int <em>maxCount</em> = 0<br>
)<br></blockquote>



In [40]:
recorded_values = pipoint.RecordedValues(
    AF.Time.AFTimeRange('*-1h','*'), 0, '', True, 100)
print([val.Value for val in recorded_values])

[13.63163, 2.018039]


<hr>
<h3>PIPoint.PlotValues Method</h3>

In [37]:
plot_values = pipoint.PlotValues(
    AF.Time.AFTimeRange('*-1d','*'), 20)
print([val.Value for val in plot_values])

[69.125, 13.63163, 0.975068, 3.373119, 23.62705, 85.45801, 99.18903, 96.30454, 76.0009, 14.38861, 0.7723272, 3.778157, 24.18557, 85.7641, 98.44351, 98.14255, 81.86028, 70.80117, <OSIsoft.AF.Asset.AFEnumerationValue object at 0x000002C2F0EC29C8>]


In [34]:
[val.Value for val in plot_values][3].EnumerationSet.Name

'System'